In [72]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

data = pd.read_csv('Prices.csv')

y = data['Price'].values
x1 = data['Speed'].values
x2 = np.log(data['HardDrive'].values)


with pm.Model() as model:

    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta1 = pm.Normal('beta1', mu=0, sigma=10)
    beta2 = pm.Normal('beta2', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=10)
    
    mu = alpha + beta1 * x1 + beta2 * x2
    
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    trace = pm.sample(2000, tune=1000, return_inferencedata=False, random_seed=42)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, sigma]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 3 seconds.


In [73]:
hdi_beta1 = az.hdi(trace['beta1'], hdi_prob=0.95)
hdi_beta2 = az.hdi(trace['beta2'], hdi_prob=0.95)

print(f"95% HDI pentru β1: {hdi_beta1}")
print(f"95% HDI pentru β2: {hdi_beta2}")

95% HDI pentru β1: [14.17661752 16.83388617]
95% HDI pentru β2: [212.81915241 237.67437428]


Coeficienții **β₁** și **β₂** determină dacă frecvența procesorului și dimensiunea hard diskului sunt predictori semnificativi pentru prețul de vânzare al unui computer. 

- Dacă un coeficient este **0**, acel predictor nu este util. 
- Dacă un interval HDI **nu include valoarea 0**, atunci predictorul respectiv este considerat **semnificativ**.

Cum **β₁** și **β₂** sunt semnificativ mai mari ca 0 si intervalul hdi nu continer valoare 0 sau valori <0, atunci acesti coeficenti sunt semnificativi pentru determinarea pretului.


In [74]:
x1_new = 33
x2_new = np.log(540)

# Calcularea valorii medii μ pentru noile date
mu_new = trace['alpha'] + trace['beta1'] * x1_new + trace['beta2'] * x2_new

# Simularea 5000 de extrageri
mu_simulated = np.random.choice(mu_new, size=5000)

# Calcularea intervalului HDI de 90% pentru prețul estimat
hdi_mu = az.hdi(mu_simulated, hdi_prob=0.90)
print(f"Intervalul HDI de 90% pentru pretul estimat: {hdi_mu}")

Intervalul HDI de 90% pentru pretul estimat: [1932.50415504 1998.99469696]


In [75]:
y_simulated = np.random.normal(mu_new[0:5000], np.random.choice(trace['sigma'], size=5000))

# Calcularea HDI
hdi_y = az.hdi(y_simulated, hdi_prob=0.90)
print(f"Intervalul HDI de 90% pentru prețul predictiv: {hdi_y}")

Intervalul HDI de 90% pentru prețul predictiv: [1452.61991091 2470.16612466]


In [ ]:
x3 = [1 if value == 'yes' else 0 for value in data['Premium']]
print(x3)

with pm.Model() as model:

    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta1 = pm.Normal('beta1', mu=0, sigma=10)
    beta2 = pm.Normal('beta2', mu=0, sigma=10)
    beta3 = pm.Normal('beta3', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=10)
    
    mu = alpha + beta1 * x1 + beta2 * x2 + beta3 * x3
    
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    trace_premium = pm.sample(2000, tune=1000, return_inferencedata=False, random_seed=42)

[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, beta3, sigma]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 4 seconds.


[17.94184204  7.89643989 13.99404741 ...  9.43323342 11.30724407
 28.41876265]


In [ ]:
hdi_beta1 = az.hdi(trace_premium['beta1'], hdi_prob=0.95)
hdi_beta2 = az.hdi(trace_premium['beta2'], hdi_prob=0.95)
hdi_beta3 = az.hdi(trace_premium['beta3'], hdi_prob=0.95)

print(f"95% HDI pentru β1: {hdi_beta1}")
print(f"95% HDI pentru β2: {hdi_beta2}")
print(f"95% HDI pentru β3: {hdi_beta3}")

# Cum beta3 e pozitiv si nu include 0, inseamna ca un calculator premium e mai scump decat unul normal

95% HDI pentru β1: [14.11638141 16.7702907 ]
95% HDI pentru β2: [209.94900633 235.21168345]
95% HDI pentru β3: [ 2.25589452 39.00264909]
